<a href="https://colab.research.google.com/github/tamal2000/Capstone-Project/blob/master/Part2%3A%20Week%203%20Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Neighborhoods of Toronto
###Part 2: Importing GEO data into the dataframe <br>Applied Data Science Capstone(Week 3 Project)
<p>Objective this part of the assignment is to import latitude and longitude of each location of the dataframe to analyze <B>Neighborhoods of Toronto</B>

####Beutiful soup librery is used below to scrap data from wikipedia page.



In [0]:
#import related libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import folium # map rendering library

# import k-means from clustering stage
from sklearn.cluster import KMeans

# import breautiful soup to scrap data from websites 
from bs4 import BeautifulSoup

##Fetch the wikipedia page and parsed using Beautiful Soup to scrap data

In [0]:
#Below codes are used to fetch the data from wikipedia by using Request method
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
res = requests.get(URL).text
# parse the link using Beautiful soup to get stracture dat 
soup = BeautifulSoup(res,'lxml')


##Preparing a two dimentional list to collect only required data

In [0]:
#Initiating Location List. This list will be used to gater all the respecive data
location_list = [['PostCode', 'Borough', 'Neighborhood']]

#A temp file is created to keep all unique PostCodes
postcode_temp = []

#PostCode dedup function is created to identify the index of existing Postcode in the location list. 
def postcode_dedup(data, search):
    for i, e in enumerate(data):
        try:
            return i, e.index(search)
        except ValueError: pass

#Below "for loop" is used to travers through the table data  
for items in soup.find('table', class_='wikitable').find_all('tr')[1::1]:
    data = items.find_all(['th','td'])
    try:
        #get post postcode, Borough and Neighbourhood from the soup file 
        Postcode = data[0].text.strip()
        Borough = data[1].text.strip()
        Neighbourhood = data[2].text.strip()
    except IndexError:pass
    
    #If Post code was found for the first time we will procceed appending the list with Postcode, Borough and Neighbourhood data
    if Postcode not in postcode_temp: 
      postcode_temp.append(Postcode) #appending ptcode data to the temporary postcode list

      #check if Brough has legitimate values, otherwise skip the entry
      if Borough != "Not assigned":
        
        #if Neighbourhood data is missing we will replace the missing data wil the Borough data 
        if Neighbourhood == "Not assigned":
          Neighbourhood = Borough
        location_list.append([Postcode, Borough, Neighbourhood])
    else:
      #finding the index to post code if its already exists and updating adding neighbourhood data with the existing neighbourhood data of that postcode
       key = postcode_dedup(location_list, Postcode)
       location_list[key[0]][2] = location_list[key[0]][2] + ', ' + Neighbourhood


##Transforming List to Pandas Dataframe

In [4]:
#importing and prparing Pandas dataset
df = pd.DataFrame(location_list, columns =['Postcode', 'Borough', 'Neighbourhood']).set_index("Postcode")
df = df[1:]

#pd.set_option('display.max_colwidth', -1)
df.head(10)

,Borough,Neighbourhood
Postcode,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront, Regent Park"
M6A,North York,"Lawrence Heights, Lawrence Manor"
M7A,Queen's Park,Queen's Park
M9A,Etobicoke,Islington Avenue
M1B,Scarborough,"Rouge, Malvern"
M3B,North York,Don Mills North
M4B,East York,"Woodbine Gardens, Parkview Hill"


In [5]:
print("The shape of dataset is: "+str(df.shape))

The shape of dataset is: (103, 2)


#Part 2 Starts here: 

Step 1: Download Geo location data to ll_data.csv file

In [6]:
!wget -O ll_data.csv https://cocl.us/Geospatial_data

--2019-10-23 19:07:37--  https://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.48.113.194
Connecting to cocl.us (cocl.us)|169.48.113.194|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-10-23 19:07:40--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.25.197, 107.152.24.197
Connecting to ibm.box.com (ibm.box.com)|107.152.25.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-10-23 19:07:40--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [followin

Step 2: Import the geo location data to df_ll dataframe

In [7]:
df_ll = pd.read_csv("ll_data.csv").set_index("Postal Code")
df_ll.head(10)

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476
M1J,43.744734,-79.239476
M1K,43.727929,-79.262029
M1L,43.711112,-79.284577
M1M,43.716316,-79.239476


Step 3: join both the data frame to assign latitude and lonitude information in the dataframe

In [11]:
#sort the dataframe first to match with geo location dataframe
df = df.sort_values(by=['Postcode'])
#join the dataframe
joined_data = df.join(df_ll)
joined_data

,Borough,Neighbourhood,Latitude,Longitude
Postcode,,,,
M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476
M1J,Scarborough,Scarborough Village,43.744734,-79.239476
M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
